# PHM 데이터셋 고장 분류

CNN-LSTM 모델을 사용하여 장비 고장 유형을 분류하는 실습 자료입니다.

## 목표
- 시계열 센서 데이터 전처리 방법 학습
- 슬라이딩 윈도우 기법 이해 및 구현
- CNN-LSTM 하이브리드 모델 구축
- 고장 분류 모델 학습 및 평가

## 1. 라이브러리 Import

In [ ]:
# 기본 라이브러리
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm

# 전처리
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

# 딥러닝
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau

# 시드 고정 (재현성)
np.random.seed(42)
tf.random.set_seed(42)

print("TensorFlow version:", tf.__version__)

## 2. PHM 데이터 로드

**NASA Turbofan Engine Degradation (C-MAPSS) 데이터셋 사용**

이 데이터셋은 항공기 터보팬 엔진의 Run-to-Failure 시뮬레이션 데이터로, 실제 PHM 연구에서 널리 사용됩니다.

### 데이터 특성
- **센서 개수**: 21개 (온도, 압력, 진동 등)
- **운영 설정**: 3개 (고도, 속도, 스로틀)
- **고장 유형**: 
  - 0: Normal (RUL > 100 cycles)
  - 1: Early Degradation (50 < RUL ≤ 100)
  - 2: Advanced Degradation (20 < RUL ≤ 50)
  - 3: Critical (RUL ≤ 20)

**Note**: 전처리된 데이터가 `../data/processed/` 폴더에 준비되어 있습니다.

In [ ]:
# PHM 데이터 로드
# Hint: pandas의 read_csv를 사용하여 '../data/processed/train_FD001_processed.csv' 파일을 로드하세요

print("NASA Turbofan Engine Degradation 데이터 로드 중...")

# 여기에 코드를 작성하세요
df = # TODO: CSV 파일 로드

print(f"\n데이터 로드 완료!")
print(f"데이터 shape: {df.shape}")

# 데이터 탐색
# Hint: df.head(), df.info(), df.describe() 등을 활용하여 데이터를 살펴보세요
# TODO: 엔진 개수 출력
# TODO: 고장 유형 분포 출력

## 3. 데이터 탐색 (EDA)

**Hint**: 데이터의 분포, 고장 유형별 특성, 센서 간 상관관계 등을 시각화하세요.

In [ ]:
# 데이터 탐색 (EDA)
# Hint: 데이터의 기본 통계, 분포, 시각화를 통해 데이터를 이해하세요

# 센서 컬럼 추출
sensor_cols = [col for col in df.columns if col.startswith('sensor_')]
print(f"센서 개수: {len(sensor_cols)}")

# TODO: df.describe()를 사용하여 센서 데이터의 기본 통계 확인

# TODO: 고장 유형별 분포 시각화
# Hint: matplotlib의 subplot을 사용하여 bar chart와 pie chart 2개를 그리세요
# - plt.figure(figsize=(12, 5))
# - plt.subplot(1, 2, 1) - bar chart
# - plt.subplot(1, 2, 2) - pie chart
# - df['fault_type'].value_counts()를 활용

## 4. 데이터 전처리

### 4.1 특징(X)과 타겟(y) 분리

In [ ]:
# 특징(X)과 타겟(y) 분리
# Hint: 센서 컬럼만 선택하여 X로, fault_type 컬럼을 y로 분리하세요

sensor_cols = [col for col in df.columns if col.startswith('sensor_')]

# TODO: X와 y 분리
X = # TODO: 센서 데이터만 선택하여 numpy array로 변환
y = # TODO: fault_type 컬럼을 numpy array로 변환

print(f"X shape: {X.shape}")
print(f"y shape: {y.shape}")
print(f"센서 개수: {len(sensor_cols)}")
print(f"고장 유형 개수: {len(np.unique(y))}")

### 4.2 데이터 정규화

**Hint**: StandardScaler를 사용하여 각 센서 데이터를 정규화하세요. 이는 학습 속도와 성능을 향상시킵니다.

In [ ]:
# 데이터 정규화
# Hint: StandardScaler를 사용하여 센서 데이터를 정규화하세요
# 정규화는 평균 0, 표준편차 1로 변환하여 학습 성능을 향상시킵니다

# TODO: StandardScaler 생성 및 fit_transform
scaler = # TODO: StandardScaler 객체 생성
X_scaled = # TODO: X에 fit_transform 적용

print(f"정규화 전 - 평균: {X.mean():.4f}, 표준편차: {X.std():.4f}")
print(f"정규화 후 - 평균: {X_scaled.mean():.4f}, 표준편차: {X_scaled.std():.4f}")

### 4.3 슬라이딩 윈도우 생성

시계열 데이터를 고정 길이의 윈도우로 분할합니다.

**Hint**: 
- `window_size`: 한 샘플에 포함될 시간 스텝 수
- `stride`: 윈도우를 이동시키는 간격
- 각 윈도우의 마지막 시점의 레이블을 사용

In [ ]:
def create_sliding_windows(X, y, window_size=50, stride=5):
    """
    슬라이딩 윈도우를 사용하여 시계열 데이터를 변환
    
    Parameters:
    - X: 입력 데이터 (n_samples, n_features)
    - y: 레이블 (n_samples,)
    - window_size: 윈도우 크기
    - stride: 윈도우 이동 간격
    
    Returns:
    - X_windows: (n_windows, window_size, n_features)
    - y_windows: (n_windows,)
    """
    
    X_windows = []
    y_windows = []
    
    # TODO: for 루프를 사용하여 stride 간격으로 윈도우를 생성하세요
    # Hint:
    # - range(0, len(X) - window_size + 1, stride)를 사용
    # - 각 윈도우는 X[i:i+window_size] 형태로 슬라이싱
    # - 레이블은 윈도우의 마지막 시점 y[i+window_size-1] 사용
    # - X_windows.append(window), y_windows.append(label)
    
    
    return np.array(X_windows), np.array(y_windows)

# 윈도우 생성
window_size = 50  # 50 사이클을 하나의 샘플로
stride = 5        # 5 사이클씩 이동

X_windows, y_windows = create_sliding_windows(X_scaled, y, window_size, stride)

print(f"윈도우 설정: window_size={window_size}, stride={stride}")
print(f"원본 데이터: {X_scaled.shape} → 윈도우 데이터: {X_windows.shape}")
print(f"생성된 윈도우 개수: {len(X_windows):,}개")

### 4.4 One-hot Encoding

**Hint**: to_categorical을 사용하여 레이블을 one-hot 벡터로 변환하세요.

In [ ]:
# One-hot Encoding
# Hint: keras.utils.to_categorical을 사용하여 정수 레이블을 one-hot 벡터로 변환하세요
# 예: 2 → [0, 0, 1, 0]

# TODO: y_windows를 one-hot 벡터로 변환
y_categorical = # TODO: to_categorical 사용

print(f"원본 레이블 shape: {y_windows.shape}")
print(f"One-hot 레이블 shape: {y_categorical.shape}")
print(f"\n예시: {y_windows[0]} → {y_categorical[0]}")

### 4.5 Train/Test Split

In [ ]:
# Train/Test 데이터 분리
# Hint: train_test_split을 사용하여 80:20 비율로 분리하세요
# stratify 파라미터를 사용하여 클래스 비율을 유지하세요

# TODO: train_test_split으로 데이터 분리
X_train, X_test, y_train, y_test = # TODO: 코드 작성

print(f"학습 데이터: {X_train.shape}")
print(f"테스트 데이터: {X_test.shape}")

## 5. CNN-LSTM 모델 구축

### 모델 아키텍처
1. **Conv1D**: 센서 간 공간적 특징 추출
2. **LSTM**: 시간적 패턴 학습
3. **Dense**: 분류

**Hint**: 
- Input shape: (window_size, n_features)
- Conv1D 레이어 후 MaxPooling 적용
- LSTM 레이어는 return_sequences=False로 설정
- 최종 Dense 레이어는 softmax 활성화 함수 사용

In [ ]:
def build_cnn_lstm_model(input_shape, n_classes):
    """
    CNN-LSTM 모델 구축
    
    아키텍처 힌트:
    1. Conv1D → BatchNorm → MaxPooling → Dropout (2~3번 반복)
       - filters: 64 → 128 → 256으로 증가
       - kernel_size: 3
       - padding: 'same'
    2. LSTM → Dropout
       - units: 128
       - return_sequences: False
    3. Dense → BatchNorm → Dropout
       - units: 64
    4. Output Dense (softmax)
       - units: n_classes
    
    Parameters:
    - input_shape: (window_size, n_features)
    - n_classes: 분류할 클래스 개수
    """
    
    model = models.Sequential()
    
    # TODO: CNN 블록들을 추가하세요
    # Hint: model.add(layers.Conv1D(...))
    
    
    # TODO: LSTM 레이어를 추가하세요
    # Hint: model.add(layers.LSTM(...))
    
    
    # TODO: Dense 레이어들을 추가하세요
    
    
    # TODO: 출력 레이어 추가
    # Hint: model.add(layers.Dense(n_classes, activation='softmax'))
    
    
    return model

# 모델 생성 및 컴파일
input_shape = (window_size, X_scaled.shape[1])
n_classes = y_categorical.shape[1]

model = build_cnn_lstm_model(input_shape, n_classes)

# TODO: 모델 컴파일
# Hint: model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


model.summary()

## 6. 모델 학습

**Hint**: 
- EarlyStopping: val_loss가 개선되지 않으면 조기 종료
- ModelCheckpoint: 최고 성능 모델 저장
- ReduceLROnPlateau: 학습률 동적 조정

In [ ]:
# 콜백 설정
# Hint: EarlyStopping, ModelCheckpoint, ReduceLROnPlateau를 사용하여 학습 최적화

# TODO: 콜백 리스트 생성
# Hint:
# - EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=True)
# - ModelCheckpoint('../models/best_model.h5', monitor='val_accuracy', save_best_only=True)
# - ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=7, min_lr=1e-7)

callbacks = # TODO: 콜백 리스트 작성

# TODO: 모델 학습
# Hint: model.fit(X_train, y_train, validation_split=0.2, epochs=100, batch_size=32, callbacks=callbacks)

history = # TODO: 학습 코드 작성

print("\n학습 완료!")
print(f"최종 학습 정확도: {history.history['accuracy'][-1]:.4f}")
print(f"최종 검증 정확도: {history.history['val_accuracy'][-1]:.4f}")

## 7. 학습 결과 시각화

In [ ]:
# 학습 곡선 시각화
# Hint: history.history에서 'loss', 'val_loss', 'accuracy', 'val_accuracy'를 추출하여 시각화하세요

# TODO: 2개의 subplot으로 Loss와 Accuracy 그래프 그리기
# Hint:
# - plt.figure(figsize=(14, 5))
# - plt.subplot(1, 2, 1) - Loss 그래프
# - plt.subplot(1, 2, 2) - Accuracy 그래프
# - plt.plot(history.history['loss'], label='Train Loss')
# - plt.plot(history.history['val_loss'], label='Val Loss')



## 8. 모델 평가

**Hint**: 테스트 데이터로 모델 성능을 평가하고, Confusion Matrix와 Classification Report를 출력하세요.

In [ ]:
# 테스트 데이터로 예측 및 평가
# Hint: model.predict()로 예측 후 np.argmax()로 클래스 추출

# TODO: 예측 수행
y_pred_prob = # TODO: model.predict(X_test)
y_pred = # TODO: np.argmax로 클래스 추출
y_true = # TODO: y_test에서 클래스 추출

# TODO: 정확도 계산
# Hint: sklearn.metrics.accuracy_score 사용
accuracy = # TODO: 정확도 계산
print(f"테스트 정확도: {accuracy:.4f}")

# TODO: Classification Report 출력
# Hint: classification_report(y_true, y_pred, target_names=...)



### 8.1 Confusion Matrix 시각화

In [ ]:
# Confusion Matrix 시각화
# Hint: confusion_matrix를 계산하고 seaborn의 heatmap으로 시각화하세요

# TODO: Confusion Matrix 계산
cm = # TODO: confusion_matrix(y_true, y_pred)

# TODO: Heatmap으로 시각화
# Hint:
# - plt.figure(figsize=(10, 8))
# - sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=..., yticklabels=...)
# - plt.title, plt.xlabel, plt.ylabel 추가


# 추가 과제: 정규화된 Confusion Matrix도 그려보세요
# Hint: cm을 각 행의 합으로 나누어 백분율로 변환



## 9. 결과 분석 및 개선 방향

**질문**:
1. 모델의 성능이 만족스러운가요?
2. 어떤 클래스가 가장 잘 분류되었나요?
3. 어떤 클래스가 혼동되기 쉬운가요?
4. 성능을 개선하기 위해 시도할 수 있는 방법은?

**개선 방법**:

### 1. 데이터 증강
- **Stride 조정**: 현재 5 → 1~3으로 감소하여 더 많은 샘플 생성
- **다른 데이터셋 추가**: FD002, FD003, FD004 데이터 활용
- **시계열 증강**: Jittering, Scaling, Time Warping

### 2. 하이퍼파라미터 튜닝
- **Window size**: 30, 50, 100 등 다양한 크기 실험
- **LSTM units**: 64, 128, 256
- **Learning rate**: 0.0001 ~ 0.01
- **Batch size**: 16, 32, 64

### 3. 모델 아키텍처
- **Bidirectional LSTM**: 양방향 시계열 패턴 학습
- **Attention Mechanism**: 중요한 시점 강조
- **Residual Connections**: 깊은 네트워크 학습

### 4. 클래스 불균형 처리
- **Class Weights**: 희소 클래스에 높은 가중치
- **SMOTE**: 오버샘플링
- **Focal Loss**: 어려운 샘플에 집중

### 5. 실습 과제
- 윈도우 크기를 변경해보고 성능 변화를 관찰하세요
- LSTM units 수를 조정해보세요
- 추가 Conv1D 레이어를 추가하거나 제거해보세요
- FD002 데이터셋으로 학습해보세요 (더 복잡한 운영 조건)

In [ ]:
# 여기에 추가 실험 코드를 작성하세요


## 10. 모델 저장 및 로드

In [ ]:
# 최종 모델 및 전처리기 저장
# Hint: model.save()로 모델 저장, pickle로 scaler와 설정 저장

# TODO: 모델 저장
# Hint: model.save('../models/final_cnn_lstm_model.h5')


# TODO: Scaler 저장 (추후 실시간 예측에 필요)
# Hint: pickle.dump(scaler, open('파일경로', 'wb'))
import pickle


# TODO: 모델 설정 저장 (윈도우 크기, 센서 개수 등)
# Hint: 딕셔너리로 설정을 만들고 pickle로 저장
config = {
    'window_size': window_size,
    'stride': stride,
    # TODO: 추가 설정 항목들
}


# 모델 로드 예시 (참고용)
# loaded_model = keras.models.load_model('../models/final_cnn_lstm_model.h5')
# with open('../models/scaler.pkl', 'rb') as f:
#     loaded_scaler = pickle.load(f)

## 완료!

수고하셨습니다. 이 노트북을 통해 CNN-LSTM 모델을 사용한 고장 분류의 전체 파이프라인을 학습하셨습니다.

추가 학습을 위해 solution 노트북을 참고하세요.